# Report 3

### by Kaitlyn Keil and Kevin Zhang
### April 2017

<a href = http://tna.europarchive.org/20110116113217/http:/www.food.gov.uk/science/dietarysurveys/dietsurveys/>UK Food Network Dataset</a>

In [2]:
""" This file contains code used to analyze the
UK Food Nutrient Database, the information for which can be found
at this link: http://tna.europarchive.org/20110116113217/http:/www.food.gov.uk/science/dietarysurveys/dietsurveys/

The following packages are required to run this code.
"""

from __future__ import print_function, division
import pandas as pd

import sys
import numpy as np
import math
import thinkstats2
import thinkplot
import matplotlib.pyplot as plt
from operator import itemgetter
from cycler import cycler
from sklearn.feature_extraction import DictVectorizer

from collections import defaultdict, Counter
import survival
import statsmodels.formula.api as smf

ImportError: No module named 'thinkstats2'

In [166]:
### This is for the other massive set

def ReadIngredients(region='African'):
    """ Takes the ingedients dataset and converts it into a Dataframe. 
    Region can be: African, EastAsian, EasternEuropean, LatinAmerican,
        MiddleEastern, NorthAmerican, NorthernEuropean, SouthAsian,
        SoutheastAsian, SouthernEuropean, WesternEuropean"""
    temp = pd.read_csv("FoodDataset/ingredients.csv") # Reads the file with a lot of NaNs and little order
    #temp.fillna(value='None')
    temp2 = temp[temp.cuisine_type==region]
    temp_dict = temp2.to_dict('index')
    
    temp_list = []
    
    for keys,values in temp_dict.items():
        td = {}
        for key,x in values.items():
            if key=='cuisine_type':
                td[key] = x
            else:
                td[x]=1
        temp_list.append(td)
        
        
    return pd.DataFrame(temp_list).fillna(0)

def corrPlot(df):
    """ Given the ingredients dataframe, selects
    the most used ingredients and creates a correlation 
    matrix from them."""
    df1 = df.ix[:, (df.columns != 'cuisine_type')]
    temp_df = df1.sum()
    mdf = temp_df[temp_df > 50].iloc[1:]
    print(mdf)
    corr_df = pd.DataFrame()
    
    for x in mdf.index:
        if x!='index1':
            corr_df[x] = df[x]
        
    corr_df['index1'] = [int(x) for x in corr_df.index.values]
    sns.heatmap(corr_df.corr())
    return corr_df

#ingredients = ReadIngredients('SoutheastAsian')

# Shows what ingredients are most/least likely to be used together.
#  For example, in African cuisine, cumin and coriander are fairly 
#  correlated, whereas vegetable oil and olive oil together are not.
#%matplotlib inline
#corr_df = corrPlot(ingredients)

In [40]:
def ReadProximates():
    df = pd.read_excel('dietary.xls', sheetname='Proximates')
    column_list = ['Water (g)']#, 'Protein (g)', 'Fat (g)', 'Carbohydrate (g)', 'Total sugars (g)']
    for column in column_list:
        new_name = column[:-4]
        df[new_name]= df[column]
        for key, value in df[column].items():
            if isinstance(value, str):
                if '(' in value:
                    df[new_name][key] = float(value[1:-1])
                elif value=='Tr':
                    df[new_name][key] = 0.0
                elif value=='N':
                    df[new_name][key] = 0.0
                elif value==np.nan:
                    df[new_name][key] = 0.0
                else:
                    df[new_name][key] = float(value)
        
    return df

In [ ]:
#tester = pd.read_excel('dietary.xls', sheetname='Proximates')
tester = ReadProximates()
# TO DO
# For cleaning: 
#  Get rid of Ns and Trs <- Done!
#  Figure out how to deal with parentheses <- Done!
#  Convert rows into floats (.astype) <- Done!
#  Get ready for lots of brackets...
#  Potentially just make different columns for the integer values

# For report:
#  Figure out if we can actually classify based on these values
#  Make a dictionary mapping from the Group codes to actual names
#  Figure out a good model
#  Do the thing
#  PCA to figure out the components we will want to look at
#  Unclassified ML to figure out natural groups?


/home/kaitlyn/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/kaitlyn/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/kaitlyn/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/kaitlyn/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to 

In [39]:
tester.Water

0       72.3
1       72.3
2       72.3
3       72.3
4       72.3
5       72.3
6       72.3
7       72.3
8       72.3
9       72.3
10      72.3
11      72.3
12      72.3
13      72.3
14      72.3
15      72.3
16      72.3
17      72.3
18      72.3
19      72.3
20      72.3
21      72.3
22      72.3
23      72.3
24      72.3
25      72.3
26      72.3
27      72.3
28      72.3
29      72.3
        ... 
3393    72.3
3394    72.3
3395    72.3
3396    72.3
3397    72.3
3398    72.3
3399    72.3
3400    72.3
3401    72.3
3402    72.3
3403    72.3
3404    72.3
3405    72.3
3406    72.3
3407    72.3
3408    72.3
3409    72.3
3410    72.3
3411    72.3
3412    72.3
3413    72.3
3414    72.3
3415    72.3
3416    72.3
3417    72.3
3418    72.3
3419    72.3
3420    72.3
3421    72.3
3422    72.3
Name: Water, dtype: float64

In [34]:
my_str ='Water (g)'
print(my_str[:-4])

Water 


In [30]:
for key, value in tester['Water (g)'].items():
    if isinstance(value, str):
        if '(' in value:
            tester['Water (g)'][key] = value[1:-1]
        elif value=='Tr':
            tester['Water (g)'][key] = '0.0'
        elif value=='N':
            tester['Water (g)'][key] = '0.0'
        
tester['Water (g)'] = tester['Water (g)'].astype(float)

0       285.0
1       287.4
2       281.7
3       258.9
4       276.0
5       270.6
6       283.2
7       281.7
8       266.1
9       291.0
10      289.2
11      290.1
12      282.0
13      282.6
14      288.9
15      279.0
16      284.1
17      279.9
18      279.0
19      290.1
20      277.5
21      284.7
22      273.6
23      241.8
24      224.4
25      236.4
26      243.0
27      213.3
28      221.1
29      142.2
        ...  
3393    259.8
3394    286.8
3395    280.5
3396    285.9
3397    224.1
3398    219.6
3399    220.2
3400    275.1
3401    278.7
3402    283.2
3403    284.1
3404    286.8
3405    286.5
3406    255.3
3407     38.4
3408    252.9
3409    274.2
3410    287.7
3411    287.7
3412    275.7
3413    277.2
3414    222.3
3415    222.3
3416    257.4
3417    260.1
3418    251.4
3419    277.8
3420    278.1
3421    259.2
3422    287.4
Name: Water (g), dtype: float64